In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys
import time
import h5py
import copy

import matplotlib.pyplot as plt
import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import procrustes

import viz
import cameras
import data_utils
import linear_model

In [2]:
# モデルのパスを指定して構築
train_dir = '../ckpt'
ckpt = tf.train.get_checkpoint_state(train_dir, latest_filename="checkpoint")

summaries_dir = os.path.join( train_dir, "log" )
model = linear_model.LinearModel(
    1024,
    2,
    True,
    False,
    True,
    64,
    1e-3,
    summaries_dir,
    False,
    dtype=tf.float32)

if ckpt and ckpt.model_checkpoint_path:
    ckpt_name = os.path.basename( ckpt.model_checkpoint_path )






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








In [32]:
with tf.Session() as sess:
    
    # モデルへweightを読み込む
    print("Loading model {0}".format( ckpt_name ))
    model.saver.restore( sess, ckpt.model_checkpoint_path )
    print("Model loaded")
    
    enc_in = np.arange(32).reshape(1,32)
    dec_out = np.arange(48).reshape(1,48)
    dp = 1.0
    
    _, _, poses3d = model.step(sess, enc_in, dec_out, dp, isTraining=False)

Loading model checkpoint-4874200
INFO:tensorflow:Restoring parameters from ../ckpt/checkpoint-4874200
Model loaded


In [25]:
poses3d

array([[-0.0394242 ,  0.02336586,  0.02405731,  0.02954462,  0.12618995,
        -0.01935147,  0.04881043,  0.12481637,  0.04060541,  0.03942417,
        -0.02336448, -0.0240575 ,  0.1481902 ,  0.13176751, -0.03242367,
         0.18603157,  0.13249151, -0.04657107,  0.3833069 ,  0.33933014,
        -0.04174181,  0.32632723,  0.1805447 , -0.03046812,  0.24835709,
         0.21106756, -0.021041  ,  0.2531911 ,  0.19681375, -0.02256853,
         0.2397516 ,  0.21001111, -0.015431  ,  0.19216438,  0.26242778,
        -0.0394512 ,  0.21380308,  0.3658872 ,  0.03567064,  0.20204845,
         0.23013574, -0.03016636,  0.18658516,  0.3752058 , -0.00718565,
         0.1809592 ,  0.3686468 ,  0.04924582]], dtype=float32)

以下草稿

In [5]:
with tf.Session() as sess:

    # モデルのパスを指定
    train_dir = '../ckpt'
    ckpt = tf.train.get_checkpoint_state(train_dir, latest_filename="checkpoint")

    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename( ckpt.model_checkpoint_path )

    # .metaからモデルを読み込む
    print("Loading model {0}".format( ckpt_name ))
    saver = tf.train.import_meta_graph(ckpt.model_checkpoint_path + '.meta')
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Model loaded") 
    
    print(sess)

Loading model checkpoint-4874200
INFO:tensorflow:Restoring parameters from ../ckpt/checkpoint-4874200
Model loaded
